# Chap3. wor2vec
## 3.4 CBOW 모델 구현

앞 절에서 CBOW모델의 학습을 위해 데이터를 준비했었다. CBOW 모델 학습을 위해 데이터를 다시 불러보자!

In [2]:
import sys
sys.path.append('..')
from common.util import preprocess, create_contexts_target, convert_one_hot

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

contexts, target = create_contexts_target(corpus, window_size=1)

vocab_size = len(word_to_id)

# contexts와 target을 one-hot vector로 변환
target = convert_one_hot(target, vocab_size)
contexts = convert_one_hot(contexts, vocab_size)

SimpleCBOW라는 이름으로 CBOW모델을 구현해보자.

In [8]:
import sys
sys.path.append('..')
import numpy as np
from common.layers import MatMul, SoftmaxWithLoss

class SimpleCBOW:
    def __init__(self, vocab_size, hidden_size):  # 초기화 메서드의 인수로 vocab_size(어휘 수)와 은닉층의 뉴런 수(hidden_size)를 받는다.
        V, H = vocab_size, hidden_size
        
        # 가중치 초기화
        W_in = 0.01 * np.random.randn(V, H).astype('f')  # 무작위 Weight 초기화, 타입은 float 32비트로 만들어준다.
        W_out = 0.01 * np.random.randn(H, V).astype('f')
        
        # 계층 생성
        self.in_layer0 = MatMul(W_in)
        self.in_layer1 = MatMul(W_in)  # 입력에서 사용하는 MatMul 계층의 수는 입력에 사용되는 단어의 수만큼 만들어야 한다.
                                       # 윈도우의 크기가 2이므로 2개의 MatMul계층을 만들어준다.
        self.out_layer = MatMul(W_out)
        self.loss_layer = SoftmaxWithLoss()
        
        # 모든 가중치와 기울기를 리스트에 모아준다.
        layers = [self.in_layer0, self.in_layer1, self.out_layer]
        self.params, self.grads = [], []  # 신경망에서 사용되는 매개변수와 기울기를 인스턴스 변수인 params와 grads 리스트에 각각 모아둔다.
        
        for layer in layers:  
            self.params += layer.params
            self.grads += layer.grads        
        
        # 인스턴스 변수에 단어의 분산 표현을 저장
        self.word_vecs = W_in
        
    # 순전파 forward 메서드 구현 
    def forward(self, contexts, target):
        h0 = self.in_layer0.forward(contexts[:, 0])  # contexts는 3차원 배열인데...
        h1 = self.in_layer1.forward(contexts[:, 1])
        h = (h0 + h1) * 0.5
        score = self.out_layer.forward(h)
        loss = self.loss_layer.forward(score, target)
        return loss
    
    # 역전파
    def backward(self, dout=1):
        ds = self.loss_layer.backward(dout)
        da = self.out_layer.backward(ds)
        da *= 0.5
        self.in_layer0.backward(da)
        self.in_layer1.backward(da)
        return None

In [7]:
contexts.shape

(6, 2, 7)

In [6]:
contexts[:, 0].shape

(6, 7)